In [1]:
import unicodedata
import sys

for code in range(sys.maxunicode+1):
    char = chr(code)
    name = unicodedata.name(char, None)
    if name and 'TRIGRAM' in name:
        print(f'U+{code:04X}\t{char}\t{name}')

U+2630	☰	TRIGRAM FOR HEAVEN
U+2631	☱	TRIGRAM FOR LAKE
U+2632	☲	TRIGRAM FOR FIRE
U+2633	☳	TRIGRAM FOR THUNDER
U+2634	☴	TRIGRAM FOR WIND
U+2635	☵	TRIGRAM FOR WATER
U+2636	☶	TRIGRAM FOR MOUNTAIN
U+2637	☷	TRIGRAM FOR EARTH
U+1D0D3	𝃓	BYZANTINE MUSICAL SYMBOL DIESIS TRIGRAMMOS OKTO DODEKATA
U+1D0D7	𝃗	BYZANTINE MUSICAL SYMBOL YFESIS TRIGRAMMOS OKTO DODEKATA


In [2]:
FIRST_TRIGRAM_CODE = 0x2630

dummy_lines = (7, 7, 7)

for i in range(8):
    char = chr(FIRST_TRIGRAM_CODE + i)
    name = unicodedata.name(char).split()[-1]
    trigram = [dummy_lines, char, name]
    print(repr(trigram) + ',')


[(7, 7, 7), '☰', 'HEAVEN'],
[(7, 7, 7), '☱', 'LAKE'],
[(7, 7, 7), '☲', 'FIRE'],
[(7, 7, 7), '☳', 'THUNDER'],
[(7, 7, 7), '☴', 'WIND'],
[(7, 7, 7), '☵', 'WATER'],
[(7, 7, 7), '☶', 'MOUNTAIN'],
[(7, 7, 7), '☷', 'EARTH'],


In [3]:
from trigrams import Trigram

for trigram in Trigram.all:
    print(trigram.char, end=' ')
    
print()

☰ ☳ ☵ ☶ ☷ ☴ ☲ ☱ 


In [4]:
LOOKUP_TABLE = [
    # ☰   ☳   ☵   ☶   ☷   ☴   ☲   ☱ 
    ( 1, 34,  5, 26, 11,  9, 14, 43), # ☰
    (25, 51,  3, 27, 24, 42, 21, 17), # ☳
    ( 6, 40, 29,  4,  7, 59, 64, 47), # ☵
    (33, 62, 39, 52, 15, 53, 56, 31), # ☶ 
    (12, 16,  8, 23,  2, 20, 35, 45), # ☷
    (44, 32, 48, 18, 46, 57, 50, 28), # ☴
    (13, 55, 63, 22, 36, 37, 30, 49), # ☲
    (10, 54, 60, 41, 19, 61, 38, 58), # ☱
]

FIRST_HEXAGRAM = 0x4DC0

In [5]:
print(' ', *(t.char for t in Trigram.all),sep='\t')

for row, tri in zip(LOOKUP_TABLE, Trigram.all):
    print(tri.char, end='\t')
    for n in row:
        print(chr(FIRST_HEXAGRAM+n-1), end='\t')
    print()

 	☰	☳	☵	☶	☷	☴	☲	☱
☰	䷀	䷡	䷄	䷙	䷊	䷈	䷍	䷪	
☳	䷘	䷲	䷂	䷚	䷗	䷩	䷔	䷐	
☵	䷅	䷧	䷜	䷃	䷆	䷺	䷿	䷮	
☶	䷠	䷽	䷦	䷳	䷎	䷴	䷷	䷞	
☷	䷋	䷏	䷇	䷖	䷁	䷓	䷢	䷬	
☴	䷫	䷟	䷯	䷑	䷭	䷸	䷱	䷛	
☲	䷌	䷶	䷾	䷕	䷣	䷤	䷝	䷰	
☱	䷉	䷵	䷻	䷨	䷒	䷼	䷥	䷹	
